### Подключение библиотек

In [ ]:
import os
import time
import re
import random
import requests
from tbselenium.tbdriver import TorBrowserDriver

### Авторизация

In [ ]:
def work():
    browser = TorBrowserDriver("./tor-browser-linux64-8.5.3_ru/tor-browser_ru/")
    browser.get('http://elibrary.ru/')

    textarea = browser.find_element_by_id('login')
    textarea.send_keys('***')
    textarea1 = browser.find_element_by_id('password')
    textarea1.send_keys('***')

    submit1 = browser.find_element_by_xpath('//div[@class="butred" and contains(text(), "Вход")]')
    submit1.click()
    
    browser.get('https://elibrary.ru/projects/subscription/rus_titles_free.asp')    
    return browser

### Получение id всех журналов

In [ ]:
browser = work()
browser.get('https://elibrary.ru/projects/subscription/rus_titles_free.asp')
parentElement = browser.find_element_by_id('restab')
elementList = parentElement.find_elements_by_tag_name('tr')

all_id = list()
for elem in elementList:    
    all_tag = elem.get_attribute('outerHTML')
    s = re.search('id="a\d{4,6}"', str(all_tag))
    if s:
        s2 = s.group(0)
        s3 = re.search('\d{4,6}', str(s2))
        if s3:
            all_id.append(s3.group(0))

### Проход по каждой статье и скачивание всех pdf-ов

In [ ]:
browser = work()

for journal_id in all_id:
    
    url = 'https://elibrary.ru/contents.asp?titleid=' + str(journal_id)
    browser.get(url)    
    parentElement1 = browser.find_element_by_id('restab')
    elementList1 = parentElement1.find_elements_by_tag_name('tr')
    
    all_articles = list()
    num_art = 0
    for index in range(len(elementList1)):
        elem = elementList1[index]
        if num_art == 10:
            browser.quit()
            time.sleep(random.randint(40,100))
            browser = work()
            url = 'https://elibrary.ru/contents.asp?titleid=' + str(journal_id)
            browser.get(url)
            parentElement1 = browser.find_element_by_id('restab')
            elementList1 = parentElement1.find_elements_by_tag_name('tr')
            elem = elementList1[index]
            num_art = 0
        num_art += 1
        all_tag = elem.get_attribute('innerHTML')
        s = re.search('id="tid\d{6,20}"', str(all_tag))
        if s:
            s2 = s.group(0)
            s3 = re.search('\d{6,10}', str(s2))
            new_id = s3.group(0)          
            
            flag_a = True
            try:
                a = browser.find_element_by_xpath('//a[@href="javascript:load_article(' + new_id + ')"]') 
            except:
                print('WRONG')
                flag_a = False
                pass
            
            #Скачивание pdf
            window_before = browser.current_window_handle  
            
            if flag_a == True:
                
                time.sleep(random.randint(5,10))
                
                a.click()     
                time.sleep(random.randint(10,17))
                
                all_windows = browser.window_handles
                for window in all_windows:
                    if window != window_before:
                        window_after = window

                browser.switch_to_window(window_after) 


                #time.sleep(5)
                flag = False
                j = 0
                while flag == False and j < 30:
                    url = browser.current_url
                    if url != 'about:blank':
                        flag = True
                    time.sleep(1)
                    j += 1
                    
                if url == 'about:blank':
                    pass
                else:
                    r = requests.get(url, stream=True)

                    #В названии должен быть указан id-журнала + id-статьи
                    dir_name = str("./Статьи/id_journal=" + journal_id)
                    if os.path.exists(dir_name) == False:
                        os.mkdir(dir_name)

                    fname = dir_name +"/id_article=" + new_id + ".pdf"
                    with open(fname, "wb") as fd:
                        for ch in r.iter_content():
                            fd.write(ch)

                browser.close()
                browser.switch_to_window(window_before)
            
    time.sleep(random.randint(0,4))
    print(journal_id) 